In [6]:
import cv2
import os
import sys
import numpy as np
import tensorflow as tf

In [7]:

DATA_PATH = 'D:\dataset\\' # Path to folder containing data

shape_to_label = {'foot':np.array([1.,0.,0.]),'notfoot':np.array([0.,1.,0.]),'ok':np.array([0.,0.,1.])}
arr_to_shape = {np.argmax(shape_to_label[x]):x for x in shape_to_label.keys()}

imgData = list()
labels = list()

for dr in os.listdir(DATA_PATH):
    if dr not in ['foot','notfoot']:
        continue
    print(dr)
    lb = shape_to_label[dr]
    i = 0
    for pic in os.listdir(os.path.join(DATA_PATH,dr)):
        path = os.path.join(DATA_PATH,dr+'/'+pic)
        img = cv2.imread(path)
        img = cv2.resize(img, (350, 300))
        imgData.append([img,lb])
        imgData.append([cv2.flip(img, 1),lb]) #horizontally flipped image
        i+=3
    print(i)

np.random.shuffle(imgData)

imgData,labels = zip(*imgData)

imgData = np.array(imgData)
labels = np.array(labels)

foot
2400
notfoot
2400


In [8]:
from keras.models import Sequential,load_model
from keras.layers import Dense,MaxPool2D,Dropout,Flatten,Conv2D,GlobalAveragePooling2D,Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet121

In [10]:
densenet = DenseNet121(include_top=False, weights='imagenet', classes=3,input_shape=(300,350,3))
densenet.trainable=True

def genericModel(base):
    model = Sequential()
    model.add(base)
    model.add(MaxPool2D())
    model.add(Flatten())
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['acc'])
    return model

dnet = genericModel(densenet)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True,
    mode='auto'
)

es = EarlyStopping(patience = 3)

history = dnet.fit(
    x=imgData,
    y=labels,
    batch_size = 16,
    epochs=6,
    callbacks=[checkpoint, es],
    validation_split=0.2
)

dnet.save_weights('model.h5')

with open("model.json", "w") as json_file:
    json_file.write(dnet.to_json())

Epoch 1/6
160/160 [==============================] - 5129s 32s/step - loss: 0.9401 - acc: 0.9102 - val_loss: 7.4079 - val_acc: 0.8219

Epoch 00001: val_acc improved from -inf to 0.82187, saving model to model.h5
Epoch 2/6
 25/160 [===>..........................] - ETA: 1:10:48 - loss: 0.0603 - acc: 0.9745

KeyboardInterrupt: 